In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
df = pd.read_csv("2025_Airbnb_NYC_listings.csv")

selected_columns_for_table = ['id','host_id','name','description','neighborhood_overview','host_about','host_is_superhost',
    'host_response_time','host_response_rate','host_acceptance_rate','host_identity_verified','host_has_profile_pic','host_location'
]

df1 = df[selected_columns_for_table]
df1.isnull().sum()

# neighborhood_overview 결측치 많아서 유무대체 
df['neighborhood_overview_exists'] = df['neighborhood_overview'].notnull().astype(int)

# name 글자수기준 중앙값으로 그룹
df['name_length'] = df['name'].fillna('').astype(str).apply(len)
med_length = df['name_length'].median()

def name_length_group(length, med):
    if length == 0:
        return 'empty'
    elif length > med:
        return 'long'
    else:
        return 'short_or_med'
    
df['name_length_group'] = df['name_length'].apply(lambda x: name_length_group(x, med_length))

# description 글자수기준(결측치 405) 평균으로 그룹
df['description_length'] = df['description'].fillna('').astype(str).apply(len)
avg_length = df['description_length'].mean()

def name_length_group(length, avg):
    if length == 0:
        return 'empty'
    elif length > avg:
        return 'long'
    else:
        return 'short_or_avg'
    
df['description_length_group'] = df['description_length'].apply(lambda x: name_length_group(x, avg_length))

# host_about (결측치8917) 평균(243) 중앙값(81) 중앙값기준으로 그룹
df['host_about_length'] = df['host_about'].fillna('').astype(str).apply(len)
med_length = df['host_about_length'].median()

def name_length_group(length, med):
    if length == 0:
        return 'empty'
    elif length > med:
        return 'long'
    else:
        return 'short_or_med'
df['host_about_length_group'] = df['host_about_length'].apply(lambda x: name_length_group(x, med_length))


#host_identity_verified/host_has_profile_pic /host_is_superhost  
# True / Flase 1과 0으로 대체 (결측치 20/20/350 0으로 대체함)
df['host_identity_verified']=df['host_identity_verified'].fillna('f').map({'t': 1, 'f': 0}).astype(int)

df['host_has_profile_pic']=df['host_has_profile_pic'].fillna('f').map({'t': 1, 'f': 0}).astype(int)

df['host_is_superhost']=df['host_is_superhost'].fillna('f').map({'t': 1, 'f': 0}).astype(int)

# host_response_time 결측치는 중앙값으로 치환후 점수
response_time_score_map = { 
    'within an hour': 4,
    'within a few hours': 3,
    'within a day': 2,
    'a few days or more': 1
}
df['host_response_time_score'] = df['host_response_time'].map(response_time_score_map)

# 2. response_time_score 컬럼의 중앙값 계산
med_score_for_fillna = df['host_response_time_score'].median()

# 3. response_time_score 컬럼의 NaN을 계산된 중앙값으로 대체 
df['host_response_time_score'] = df['host_response_time_score'].fillna(med_score_for_fillna)

# host_response_time 칼럼에는 여전히 nan값 존재함
# response_time_score 칼럼만 중앙값대체 


# host_response_rate 컬럼 %제외하고 중앙값으로 대체
df['host_response_rate'] = df['host_response_rate'].astype(str).str.replace('%', '').astype(float)/100
med_rate2 = df['host_response_rate'].median()
df['host_response_rate']= df['host_response_rate'].fillna(med_rate2)

# 4그룹으로 나눠 점수
conditions = [
    (df['host_response_rate'] <= 0.25),
    (df['host_response_rate'] > 0.25) & (df['host_response_rate'] <= 0.5),
    (df['host_response_rate'] > 0.5) & (df['host_response_rate'] <= 0.75),
    (df['host_response_rate'] > 0.75)
]

choices = [1, 2, 3, 4]

df['host_response_rate_score'] = np.select(conditions, choices)


# host_acceptance_rate 칼럼도 %제외하고 중앙값으로 대체 
df['host_acceptance_rate'] = df['host_acceptance_rate'].astype(str).str.replace('%', '').astype(float)/100
med_rate = df['host_acceptance_rate'].median()
df['host_acceptance_rate']= df['host_acceptance_rate'].fillna(med_rate)

conditions = [
    (df['host_acceptance_rate'] <= 0.25),
    (df['host_acceptance_rate'] > 0.25) & (df['host_acceptance_rate'] <= 0.5),
    (df['host_acceptance_rate'] > 0.5) & (df['host_acceptance_rate'] <= 0.75),
    (df['host_acceptance_rate'] > 0.75)
]

choices = [1, 2, 3, 4]

df['host_acceptance_rate_score'] = np.select(conditions, choices)

# host_location 칼럼 
# host_loc 존재?
df['host_location_boolean'] = df['host_location'].notnull().astype(int)
# host_loc in NY?
df['host_location_ny'] = df['host_location'].str.contains('New York', na=False).astype(int)



selected_columns_for_table2 = ['id','host_id','name','name_length','name_length_group','description','description_length',
    'description_length_group','neighborhood_overview','host_about','host_about_length','host_about_length_group','host_is_superhost',
    'host_has_profile_pic','host_identity_verified','host_response_time','host_response_time_score','host_response_rate',
    'host_response_rate_score','host_acceptance_rate','host_acceptance_rate_score','host_location','host_location_ny'
]
df_host = df[selected_columns_for_table2]


In [8]:
df_host

,id,host_id,name,name_length,name_length_group,description,description_length,description_length_group,neighborhood_overview,host_about,...,host_has_profile_pic,host_identity_verified,host_response_time,host_response_time_score,host_response_rate,host_response_rate_score,host_acceptance_rate,host_acceptance_rate_score,host_location,host_location_ny
0,36121,62165,Lg Rm in Historic Prospect Heights,34,short_or_med,Cozy space share in the heart of a great neigh...,215,short_or_avg,Full of tree-lined streets and beautiful brown...,I’m an urban planner working for an internatio...,...,1,1,NaN,4.0,1.00,4,0.91,4,"New York, NY",1
1,36647,157798,"1 Bedroom & your own Bathroom, Elevator Apartment",49,long,Private bedroom with your own bathroom in a 2 ...,117,short_or_avg,"Manhattan, SE corner of 2nd Ave/ E. 110th street",NaN,...,1,1,NaN,4.0,1.00,4,1.00,4,"New York, NY",1
2,38663,165789,Luxury Brownstone in Boerum Hill,32,short_or_med,"Beautiful, large home in great hipster neighbo...",84,short_or_avg,"diverse, lively, hip, cool: loaded with restau...",I am a lawyer and work as an executive at an a...,...,1,1,within a few hours,3.0,1.00,4,0.40,2,"New York, NY",1
3,38833,166532,Spectacular West Harlem Garden Apt,34,short_or_med,This is a very large and unique space. An inc...,353,short_or_avg,West Harlem is now packed with great restauran...,I have been a New Yorker for a long time\n and...,...,1,1,within an hour,4.0,1.00,4,0.97,4,"New York, NY",1
4,39282,168525,“Work-from-home” from OUR home.,31,short_or_med,*Monthly Discount will automatically apply <br...,494,long,THE NEIGHBORHOOD:<br />Our apartment is locate...,I am a music producer. And my wife is a hair s...,...,1,1,within an hour,4.0,1.00,4,1.00,4,"New York, NY",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22303,1366333532374850165,40019013,Beautiful 1-Bed Apt in Harlem!,30,short_or_med,Welcome to your temporary home in the vibrant ...,508,long,NaN,NaN,...,1,1,NaN,4.0,1.00,4,0.91,4,"New York, NY",1
22304,1366717321390111215,483056418,Private Room w/ Ensuite Bath H,30,short_or_med,Stylish Private Rooms w/ En-Suite Baths in Bro...,444,long,NaN,Are you tired of the tedious and time-consumin...,...,1,1,within an hour,4.0,1.00,4,0.80,4,"New York, NY",1
22305,1366721904709517353,30283594,2 Bedroom on East Side,22,short_or_med,Located in the Murray Hill area and occupies a...,483,long,NaN,NaN,...,1,1,within an hour,4.0,0.94,4,0.99,4,"Hawthorne, NJ",0
22306,1366722692755341871,407304997,Stylish 2Bd near Bryant Park,28,short_or_med,Enjoy everything the city has to offer while l...,366,long,NaN,"At Boomerang, we believe in the idea ""you get ...",...,1,1,within an hour,4.0,0.89,4,1.00,4,"New York, NY",1


In [ ]:
selected_columns_for_table3 = ['id','host_id','name_length_group','description_length_group','neighborhood_overview','host_about_length_group','host_is_superhost',
    'host_has_profile_pic','host_identity_verified','host_response_time_score','host_response_rate_score','host_acceptance_rate_score','host_location_ny','host_location_boolean'
]